## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from dotenv import load_dotenv
load_dotenv()
google_api_key = os.getenv("google_api_key")

# Configure gmaps API key
gmaps.configure(api_key=google_api_key)

In [2]:
# 1. Import the WeatherPy_database.csv file.
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather
0,0,Bethel,US,41.3712,-73.4140,47.75,89,0,0.00,clear sky
1,1,Jalor,IN,25.3500,72.6167,94.10,15,82,12.12,broken clouds
2,2,Torbay,CA,47.6666,-52.7314,23.58,65,20,27.63,few clouds
3,3,Hermanus,ZA,-34.4187,19.2345,64.44,79,20,8.23,few clouds
4,4,Vestmannaeyjar,IS,63.4427,-20.2734,33.89,71,100,30.27,snow


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is minimum temperature you would like to vacation in?"))
max_temp = float(input("What is the maximum temperature you would like to vacation in?"))

What is minimum temperature you would like to vacation in?18
What is the maximum temperature you would like to vacation in?30


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) 
                                       & (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City ID       54
City          54
Country       54
Lat           54
Lng           54
Max Temp      54
Humidity      54
Cloudiness    54
Wind Speed    54
Weather       54
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df[preferred_cities_df['Max Temp']!='']
clean_preferred_cities_df.head(15)

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather
2,2,Torbay,CA,47.6666,-52.7314,23.58,65,20,27.63,few clouds
8,8,Yar-Sale,RU,66.8333,70.8333,22.33,95,100,14.79,light snow
43,43,Svetlogorsk,BY,52.6333,29.7333,29.41,64,15,10.83,few clouds
45,45,Vadso,NO,70.0744,29.7487,26.76,80,0,14.97,clear sky
70,70,Usinsk,RU,65.9939,57.5281,19.92,89,100,14.99,overcast clouds
107,107,Grindavik,IS,63.8424,-22.4338,28.87,80,75,16.11,light snow
148,148,Vardo,NO,70.3705,31.1107,29.14,81,8,25.34,clear sky
152,152,Leh,IN,34.1667,77.5833,26.44,68,14,2.91,few clouds
157,157,Abagur,RU,53.7308,87.2526,24.22,88,100,2.59,light snow
177,177,Ostrovnoy,RU,68.0531,39.5131,29.91,88,53,15.48,broken clouds


In [7]:
clean_preferred_cities_df.loc[clean_preferred_cities_df['Max Temp'] == ''].count()

City ID       0
City          0
Country       0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Weather       0
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng", "Weather"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Weather,Hotel Name
2,Torbay,CA,23.58,47.6666,-52.7314,few clouds,
8,Yar-Sale,RU,22.33,66.8333,70.8333,light snow,
43,Svetlogorsk,BY,29.41,52.6333,29.7333,few clouds,
45,Vadso,NO,26.76,70.0744,29.7487,clear sky,
70,Usinsk,RU,19.92,65.9939,57.5281,overcast clouds,
107,Grindavik,IS,28.87,63.8424,-22.4338,light snow,
148,Vardo,NO,29.14,70.3705,31.1107,clear sky,
152,Leh,IN,26.44,34.1667,77.5833,few clouds,
157,Abagur,RU,24.22,53.7308,87.2526,light snow,
177,Ostrovnoy,RU,29.91,68.0531,39.5131,broken clouds,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": google_api_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    params["location"] = f"{lat},{lng}"
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found..skipping.")

Hotel not found..skipping.
Hotel not found..skipping.
Hotel not found..skipping.
Hotel not found..skipping.
Hotel not found..skipping.
Hotel not found..skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df['Hotel Name']!='']
clean_hotel_df.head(15)

,City,Country,Max Temp,Lat,Lng,Weather,Hotel Name
2,Torbay,CA,23.58,47.6666,-52.7314,few clouds,Gallows Cove
8,Yar-Sale,RU,22.33,66.8333,70.8333,light snow,Yalemd
43,Svetlogorsk,BY,29.41,52.6333,29.7333,few clouds,Hotel Svetlogorsk JBIK
45,Vadso,NO,26.76,70.0744,29.7487,clear sky,Scandic Vadsø
70,Usinsk,RU,19.92,65.9939,57.5281,overcast clouds,Dobro Pozhalovaty
107,Grindavik,IS,28.87,63.8424,-22.4338,light snow,Northern Light Inn & Max's Restaurant
148,Vardo,NO,29.14,70.3705,31.1107,clear sky,Vardø Hotel AS
152,Leh,IN,26.44,34.1667,77.5833,few clouds,Hotel Ladakh Greens
157,Abagur,RU,24.22,53.7308,87.2526,light snow,Relax Deep Hotel
195,Boden,SE,24.80,65.8252,21.6886,few clouds,Quality Hotel Bodensia


In [11]:
clean_hotel_df.loc[clean_hotel_df['Hotel Name'] == ''].count()

City          0
Country       0
Max Temp      0
Lat           0
Lng           0
Weather       0
Hotel Name    0
dtype: int64

In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather</dt><dd>{Weather}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))